# Download the Ants vs Bees dataset

In [ ]:
%%bash
wget -q https://download.pytorch.org/tutorial/hymenoptera_data.zip
unzip -q hymenoptera_data.zip
ls -l hymenoptera_data

# Start with the usual imports and config

In [ ]:
import time
import os
import copy
import torch as pt
import numpy as np
import matplotlib.pyplot as plt

device = pt.device("cuda:0" if pt.cuda.is_available() else "cpu")

In [ ]:
from torchvision import transforms

mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}
data_transforms

In [ ]:
from torchvision import datasets

data_dir = 'hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: pt.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=os.cpu_count())
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

class_names

# Declare the `imshow` method to preview an image tensor

In [ ]:
def imshow(inp, title):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.title(title)
    plt.show()

# Use `torchvision.utils` and `imshow` to preview the training data
* **TODO:** Make sure you can render at least 4 images as the preview

# Declare a re-usable `train` method

In [ ]:
def train(model, loss_fn, optimizer, scheduler = None, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with pt.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = pt.max(outputs, 1)
                    loss = loss_fn(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += pt.sum(preds == labels.data)

            if phase == 'train' and scheduler is not None:
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase}: Epoch {epoch}/{num_epochs - 1} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Use the `resnet18` model from `torchvision.models` for this exercise
* **TODO:** Make sure that you download the `pretrained=True` version of the model

# Set `requires_grad == False` to freeze the model parameters so that the gradients are not computed in `backward()`


# Figure out the number of input features in the last `nn.Linear` layer of the model

# Replace the last layer of the model with a new `nn.Linear` layer for the classifier

# Check that parameters of newly constructed modules have requires_grad=True by default


# Move the transfer learning model to the GPU

# Use the `train` method to transfer learn the model

* use the `CrossEntropyLoss`
* use `SGD` only on the parameters of the last `Linear` layer
* use `SGD` learning rate of `0.001` and momentum of `0.9`